---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [87]:
import networkx as nx
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [74]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [85]:
def graph_identification():
    
    
    return ['PA','SW_L','SW_L','PA','SW_H']

In [4]:
#(Average Clustering,Average Shortest Path Length)

[(0.03167539146454044, 4.099161161161161),
 (0.5642419635919628, 5.089871871871872),
 (0.4018222222222227, 9.378702269692925),
 (0.03780379975223251, 3.1048046283934134),
 (0.0033037037037037037, 5.0785509568313305)]

In [84]:
[max(set(nx.degree(P1_Graphs[i]).values())) for i in range(5)]

[100, 13, 6, 92, 11]

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [89]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [70]:
def salary_predictions():
    
    df_all = pd.DataFrame(index=G.nodes())
    df_all['ManagementSalary'] = pd.Series(nx.get_node_attributes(G,'ManagementSalary'))
    df_all['Degree'] = pd.Series(G.degree())
    df_all['Clustering'] = pd.Series(nx.clustering(G))
    df_all['ClosenessCentrality'] = pd.Series(nx.closeness_centrality(G,normalized=True))
    df_all['BetweennessCentrality'] = pd.Series(nx.betweenness_centrality(G,normalized=True))
    df_all['PageRank'] = pd.Series(nx.pagerank(G))
    
    X_train = df_all[np.isnan(df_all['ManagementSalary']) == False].drop('ManagementSalary',axis=1)
    y_train = df_all[np.isnan(df_all['ManagementSalary']) == False]['ManagementSalary']
    
    X_test = df_all[np.isnan(df_all['ManagementSalary'])].drop('ManagementSalary',axis=1)
    
    model = LogisticRegression()
    model.fit(X_train,y_train)
    pre = model.predict_proba(X_test)[:,1]
    
    return pd.Series(pre,index=X_test.index)

In [71]:
salary_predictions()

1       0.173090
2       0.575716
5       0.960961
8       0.131396
14      0.386016
18      0.206924
27      0.284603
30      0.341153
31      0.198478
34      0.141805
37      0.129768
40      0.258249
45      0.174712
54      0.238679
55      0.186143
60      0.232928
62      0.994455
65      0.480402
77      0.081817
79      0.140420
97      0.100228
101     0.079913
103     0.208504
108     0.134309
113     0.490956
122     0.076820
141     0.434350
142     0.896282
144     0.068488
145     0.367122
          ...   
913     0.068115
914     0.091846
915     0.016076
918     0.128258
923     0.058803
926     0.084834
931     0.069112
934     0.030853
939     0.029897
944     0.015981
945     0.055965
947     0.113849
950     0.119897
951     0.062051
953     0.049077
959     0.015808
962     0.016692
963     0.144821
968     0.091603
969     0.084650
974     0.074848
984     0.029695
987     0.089987
989     0.091462
991     0.091243
992     0.017020
994     0.015765
996     0.0159

In [36]:
sorted(nx.betweenness_centrality(G,k=1000).items(),key=lambda x:x[1],reverse=True)[:10]

[(160, 0.08719326842211826),
 (86, 0.03789391598162558),
 (5, 0.031059869278112176),
 (82, 0.027947733440206308),
 (121, 0.02778712253114219),
 (107, 0.02439650221761858),
 (13, 0.02353412943538449),
 (377, 0.023252615705214568),
 (62, 0.022562325552478567),
 (64, 0.021992643249225924)]

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [88]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [105]:
def new_connections_predictions():
    
    df_all = future_connections.copy()
    df_all['JaccardCoefficient'] = [x[2] for x in nx.jaccard_coefficient(G,df_all.index)]
    df_all['PreferentialAttachment'] = [x[2] for x in nx.preferential_attachment(G,df_all.index)]
    df_all['CommunityResourceAllocation'] = [x[2] for x in nx.ra_index_soundarajan_hopcroft(G,df_all.index,community='Department')]
    df_all['CommunityCommonNeighbors'] = [x[2] for x in nx.cn_soundarajan_hopcroft(G,ebunch=df_all.index,community='Department')]
    
    df_no_nan = df_all[np.isnan(df_all['Future Connection']) == False]
    X_train = df_no_nan.drop('Future Connection',axis=1)
    y_train = df_no_nan['Future Connection']
    
    X_test = df_all[np.isnan(df_all['Future Connection'])].drop('Future Connection',axis=1)
    
    model = LogisticRegression()
    model.fit(X_train,y_train)
    pre = model.predict_proba(X_test)[:,1]
    
    return pd.Series(pre,index=X_test.index)

In [91]:
len(future_connections[np.isnan(future_connections['Future Connection'])])

122112

In [ ]:
new_connections_predictions()

(107, 348)    0.031605
(542, 751)    0.011727
(20, 426)     0.625933
(50, 989)     0.011903
(942, 986)    0.012094
(324, 857)    0.011879
(13, 710)     0.153492
(19, 271)     0.111876
(319, 878)    0.011964
(659, 707)    0.011745
(49, 843)     0.012020
(208, 893)    0.011787
(377, 469)    0.006138
(405, 999)    0.021317
(129, 740)    0.018591
(292, 618)    0.019189
(239, 689)    0.011976
(359, 373)    0.008103
(53, 523)     0.105244
(276, 984)    0.012010
(202, 997)    0.012047
(604, 619)    0.042535
(270, 911)    0.011973
(261, 481)    0.071583
(200, 450)    0.997571
(213, 634)    0.011673
(644, 735)    0.040613
(346, 553)    0.011244
(521, 738)    0.010447
(422, 953)    0.018427
                ...   
(672, 848)    0.011973
(28, 127)     0.965599
(202, 661)    0.011172
(54, 195)     0.999898
(295, 864)    0.011851
(814, 936)    0.011700
(839, 874)    0.012094
(139, 843)    0.011866
(461, 544)    0.009914
(68, 487)     0.009822
(622, 932)    0.011809
(504, 936)    0.016330
(479, 528) 